# とりあえず実践あるのみ

In [1]:
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, Multiply, Conv1D, Add, Activation, Dense, Flatten
from keras.regularizers import l2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import librosa
import librosa.display
import glob
%matplotlib inline

ImportError: Traceback (most recent call last):
  File "/Users/suzukidaishi/.pyenv/versions/3.6.2/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/Users/suzukidaishi/.pyenv/versions/3.6.2/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/Users/suzukidaishi/.pyenv/versions/3.6.2/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/Users/suzukidaishi/.pyenv/versions/3.6.2/lib/python3.6/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/Users/suzukidaishi/.pyenv/versions/3.6.2/lib/python3.6/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: dlopen(/Users/suzukida

In [ ]:
def filePianoAndGuitar(maxnum=37) :
    p = []
    g = []
    fss = []
    for i in range(1, maxnum+1):
        x, fs1 = librosa.audio.load(f"piano2guitar/piano/{str(i).zfill(3)}.wav")
        p.append(x)
        x, fs2 = librosa.audio.load(f"piano2guitar/guitar/{str(i).zfill(3)}.wav")
        g.append(x)
        fss.append(np.array([fs1, fs2]))
    return np.array(p), np.array(g), np.array(fss)

In [ ]:
p, g, fs = filePianoAndGuitar()
print(p.shape)
print(g.shape)
if np.sum(fs-np.max(fs)):print("Warning: data error")

In [ ]:
def wavenetModel(input_size) :
    filter_count = 10
    output_count = 1
    def residual_block(i):
        def f(x):
            original_x = x
            tanh_out = Conv1D(filter_count, kernel_size=2,
                              strides=1, padding="causal",
                              dilation_rate=2**i, activation="tanh",
                              use_bias=False, kernel_regularizer=l2(0.))(x)
            sigm_out = Conv1D(filter_count, kernel_size=2,
                              strides=1, padding="causal",
                              dilation_rate=2**i, activation="sigmoid",
                              use_bias=False, kernel_regularizer=l2(0.))(x)
            x = Multiply()([tanh_out, sigm_out])
            res_x = Conv1D(filter_count, 1, padding="same", use_bias=False,
                           kernel_regularizer=l2(0.))(x)
            skip_x = Conv1D(filter_count, 1, padding="same", use_bias=False,
                            kernel_regularizer=l2(0.))(x)
            res_x = Add()([original_x, res_x])
            return res_x, skip_x
        return f
    input = Input(shape=(input_size))
    out = input
    out = Reshape((input_size, 1))(out)
    skip_connections = []
    out = Conv1D(filter_count, 2, dilation_rate=1, padding="causal")(out)
    for _ in range(4):
        for i in range(0, 9+1):
            out, skip_out = residual_block(i)(out)
            skip_connections.append(skip_out)
    out = Add()(skip_connections)
    out = Activation("relu")(out)
    out = Conv1D(filter_count, 1, padding="same", kernel_regularizer=l2(0.))(out)
    out = Activation("relu")(out)
    out = Conv1D(filter_count, 1, padding="same")(out)
    out = Conv1D(output_count, 1, padding="same", activation="tanh")(out)
    model = Model(input, out)
    return model

In [ ]:
def createWave(w) :
    m = (max(np.abs(np.max(w)), np.abs(np.min(w))))
    w = (w/m)
    return w

In [ ]:
piano = createWave(p)
piano = piano.reshape(p.shape[0]*50, int((p.shape[0]*p.shape[1])/(p.shape[0]*50)))
guitar = createWave(g)
guitar = guitar.reshape(g.shape[0]*50, int((g.shape[0]*g.shape[1])/(g.shape[0]*50)))
print(piano.shape, guitar.shape)

In [ ]:
model = wavenetModel(int((p.shape[0]*p.shape[1])/(p.shape[0]*50)))
model.compile(loss="mse", optimizer="adam")

In [ ]:
history = model.fit(x=piano[:-200], y=guitar[:-200], validation_data=(piano[-200:], guitar[-200:]), epochs=10)

In [ ]:
out_x = piano[-200:].flatten()
out_y = model.predict(piano[-200:]).flatten()
ans_y = guitar[-200:].flatten()
print(out_x.shape, out_y.shape)
plt.plot(ans_y)
plt.plot(out_y)

In [ ]:
librosa.output.write_wav('ansy.wav', guitar[-200:].flatten()*(max(np.abs(np.max(g)), np.abs(np.min(g)))), fs[0,0])
librosa.output.write_wav('outy.wav', out_y*(max(np.abs(np.max(p)), np.abs(np.min(p)))), fs[0,0])
model.save('model.h5')
plt.plot(history.history['loss'],"o-",label="loss")
plt.plot(history.history['val_loss'],"o-",label="val_loss")